In [1]:
# the training dataset, in this first case, tiny shakespear dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-18 21:25:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: 'input.txt'

     0K .......... .......... .......... .......... ..........  4% 3,23M 0s
    50K .......... .......... .......... .......... ..........  9% 6,28M 0s
   100K .......... .......... .......... .......... .......... 13% 11,9M 0s
   150K .......... .......... .......... .......... .......... 18% 12,1M 0s
   200K .......... .......... .......... .......... .......... 22% 12,0M 0s
   250K .......... .......... .......... .......... .......... 27% 12,0M 0s
   300K .......... .......... .......... .......... .......... 32% 11,1M 0s
   350K .......... ..

In [2]:
# read it in to inspect it
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [6]:
# the first 1000 characters
print(text[:1_000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [11]:
# all unique characters that occur in this text
# set: takes unique characters, list: creates a list of these unique characters ,sorted: sorts this list of unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [31]:
stoi = { ch:i for i, ch in enumerate(chars) } # creates a dictionary where each characters index in our sorted chars list is the value and the character is the key, e.g {"A": 13}
itos = { i:ch for i, ch in enumerate(chars) } # the other way around, e.g. {13: "A"}
encode = lambda s: [stoi[c] for c in s] # for character in stoi, return the value of the key
decode = lambda l: "".join([itos[i] for i in l]) # other way around, concatenate to string

hello_world = encode("hello world")
print(hello_world) # [46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]

decoded_hello_world = decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42])
print(decoded_hello_world) # hello world

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [35]:
# let's enode the entire text dataset and store it into a torch.Tensor
import torch # using pytorch
data = torch.tensor(encode(text), dtype=torch.long) # text = all out data
print(data.shape, data.dtype)
print(data[:1000]) # this is essentially a copy of our previous 1000 character printout, but now in integers

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [44]:
# lets split our dataset into a train/val set
n = int(0.9*len(data)) # 90% of the length of our dataset
train_data = data[:n] # our data from 0 to 1003854
val_data = data[n:] # our data from 1003854 to 1115394
print(f"train data length : {len(train_data)} | val data length : {len(val_data)}")

train data length : 1003854 | val data length : 111540


In [53]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

[18, 47, 56, 57, 58,  1, 15, 47, 58]
What this block of training data will "teach" our gpt model is:
in the context of 18 : 47 is most likely to come next, in the context of 18, 47 : 56 is likely to come next. in the context of 18, 47, 56, 57 : 1 is likely to come next etc.


In [58]:
x = train_data[:block_size] # tensor([18, 47, 56, 57, 58,  1, 15, 47])
y = train_data[1:block_size+1] # tensor([47, 56, 57, 58,  1, 15, 47, 58])
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [62]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train") #x batch, y batch
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("-------")


for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44,

In [64]:
print(xb) # our inpit to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [84]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off th elogits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) batch: 8, time: 4, channel: 65 (vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [85]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [90]:
batch_size = 32
for steps in range(10_000):
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(loss.item())
    

2.4461376667022705
2.427589178085327
2.4640743732452393
2.4112191200256348
2.4496371746063232
2.438636541366577
2.5585086345672607
2.515166997909546
2.5514657497406006
2.338054895401001
2.410571336746216
2.460509777069092
2.598633050918579
2.598629951477051
2.424546480178833
2.5031988620758057
2.4596126079559326
2.4940969944000244
2.4237325191497803
2.5872318744659424
2.4127373695373535
2.5548806190490723
2.4255197048187256
2.4538261890411377
2.5039048194885254
2.4494566917419434
2.406545639038086
2.4326553344726562
2.511512041091919
2.533144950866699
2.541266918182373
2.557629108428955
2.46083927154541
2.3495402336120605
2.5304923057556152
2.605462074279785
2.3500540256500244
2.388798475265503
2.5156750679016113
2.577136516571045
2.283259868621826
2.445627450942993
2.388678789138794
2.3968262672424316
2.5455000400543213
2.457970380783081
2.433074951171875
2.4418435096740723
2.421562433242798
2.375838279724121
2.437729835510254
2.6002697944641113
2.453763246536255
2.455972909927368
2.5

In [89]:
# 10_000 steps
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



BRS:

THAnrt t fa boun-s trconnou

No: ENGUS:
tepare ofo.'s ne:
We Prellothe, s;
NE: t an re, belono


## The mathematical trick in self-attention

In [91]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [94]:
# we want z[b, t] = mean_{i<=t} x[b, i]

xbow = torch.zeros((B, T, C)) # x bag of words, initialized at 0 
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [100]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [98]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10,(3, 2)).float()
c = a @ b # matrix multiplication

print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)


a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
